In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
from openai import OpenAI
import requests
import json
import re
import numpy as np
import random
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
from tqdm.notebook import tqdm
from itertools import combinations
import pandas as pd

import sys
sys.path.append("../source")
import helpers
import data
import models
import api

import importlib
importlib.reload(models)

client = OpenAI()
egp = helpers.get_egp()

[nltk_data] Downloading package punkt to
[nltk_data]     /cluster/scratch/dglandorf/cache...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Ask GPT4 for regular expressions of the corpus search.

In [95]:
# sample rule
rule = egp.sample(1).iloc[0]
# compose prompt
prompt = f"""I want to find sentences with {" ".join(rule['Can-do statement'].split(" ")[2:]).replace(".","")} (category: {rule['SuperCategory']}: {rule['SubCategory']}) a text.
What are Python regular expressions I should search for?
Return a json array with basic regex: {{ "search": [regex1, regex2, ...] }}."""
print(prompt)
# query API
messages = [{ "role": "user", "content": prompt}]
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",#"gpt-3.5-turbo-1106",#os.getenv("OPENAI_DEFAULT_MODEL"),
    messages=messages,
    response_format={ "type": "json_object" },
    n=1,
    max_tokens=512,
)
#print(response.choices[0].message.content)

matched_strings = set()
all_expressions = set()
strings_to_search = rule['Example'].split("\n\n")

for choice in response.choices:
    all_expressions.update(json.loads(choice.message.content)['search'])
for pattern in all_expressions:
    print(pattern)

for string in strings_to_search:
    for pattern in all_expressions:
        if re.search(pattern.replace("\x08", "").replace("\x09", "").replace("++", "+"), string, re.IGNORECASE):
            matched_strings.add(string)
            break

print(rule['Example'])

print(len(matched_strings)/len(strings_to_search))

other_rule = egp.sample(1).iloc[0]
string_prob_not_to_match = other_rule['Example'].split("\n\n")
for string in string_prob_not_to_match:
    for pattern in all_expressions:
        if re.search(pattern.replace("\x08", "").replace("\x09", "").replace("++", "+"), string, re.IGNORECASE):
            print(string)
            break


I want to find sentences with the singular reflexive pronouns 'myself', 'yourself', 'himself' and 'herself' for emphasis (category: PRONOUNS: reflexive) a text.
What are Python regular expressions I should search for?
Return a json array with basic regex: { "search": [regex1, regex2, ...] }.
\bhimself\b
\byourself\b
\bmyself\b
\bherself\b
The best solution will be if you decide yourself what to do. 

I think it was the director himself who guided the actress through our wonderful school gardens to film the opening scene there. 

The food was delicious, especially the cake, which Sandra had cooked herself.
1.0
I myself, having been educated the hard way, would specifically insist on them getting the best possible school education. 


In [2]:
def get_response(prompt, stop=[], max_tokens=64, model=None):
    response = client.chat.completions.create(
        model=model if model else "gpt-4o",
        messages=[{ "role": "user", "content": prompt}],
        temperature=0,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        n=1,
        stop=stop
    )
    return response.choices[0].message.content

In [3]:
def describe_skill(rule):
    return f"""Learn this grammar skill from the description and the examples:
{rule['guideword']}: {rule['Can-do statement']}
Category: {rule['SuperCategory']}
SubCategory: {rule['SubCategory']}
CEFR Level: {rule['Level']}
{f"Lexical Range: {int(rule['Lexical Range'])}/3" if not np.isnan(rule['Lexical Range']) else ""}
Examples:
{rule['Example']}"""

In [16]:
def get_words(rule):
    prompt = f"""
{describe_skill(rule)}

Which words and combinations of words should I search for in a text to find other example sentences of this grammar pattern? Only output the words as a list.
"""
    response = get_response(prompt)
    words = [res.replace("- ", "") for res in response.split("\n")]
    words = [word for word in words if word != ""]
    return words

In [5]:
def get_questions(rule):
    prompt = f"""
{describe_skill(rule)}

Create a checklist with questions that all necessarily need to be answered with "Yes" to judge whether that skill is used in a sentence. Only return the enumerated questions."""

    return get_response(prompt, max_tokens=256)

In [6]:
def detect_prompt(sentence, questions):  
    return f"""
Sentence:
{sentence}

Questions:
{questions}

Given the sentence, answer the given questions with either a clear "Yes" or "No". Only return the enumerated responses.
"""

In [7]:
def judge(sentences, questions, verbose=True, max_positives=25, max_negatives=25):
    positives = []
    negatives = []
    for probe in tqdm(sentences):
        if verbose: print(probe)
        response = get_response(detect_prompt(probe, questions))
        if verbose: print(response)
        if re.search(r"\bNo\b", response):
            negatives.append(probe)
        else:
            positives.append(probe)
        if len(positives) > max_positives: break
        if len(negatives) > max_negatives: break
    return positives, negatives

In [ ]:
def search_corpus(words, sents):
    results = {}
    for sentence in tqdm(sents):
        count = 0
        for word in words:
            if bool(re.search(r"\b" + re.escape(word.lower()) + r"\b", sentence.lower())):
                count = count/2 + 1 + word.count(' ') / 4 + int(word in sentence) / 8
        results[sentence] = count

    hits = [key for key, value in results.items() if value > 0]
    sorted_sentences = sorted(hits, key=lambda x: results[x], reverse=True)
    return results, sorted_sentences

In [19]:
# model training
def get_trained_classifier(positive, negative, others, classifier, ratio=1, verbose=False):
    dataset = data.get_dataset(positive, negative, others, models.bert_tokenizer, 64, ratio*len(positive)/len(negative), verbose=verbose) 
    train_dataloader, val_dataloader = data.get_loaders(dataset)
    _, val_metrics = models.train(classifier, train_dataloader, val_dataloader, num_epochs=None, verbose=verbose, leave=not verbose)
    return classifier, val_metrics

In [8]:
sents = data.get_mixed_sentences(1000000)
sents = set(sents) # remove duplicates

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:32<00:00,  8.16s/it]


In [9]:
max_batches = 16
batch_size = 256
sentences = list(sents)
encoded_inputs = models.bert_tokenizer(sentences[:12*max_batches*batch_size], return_tensors='pt', max_length=64, padding='max_length', truncation=True)
encoded_inputs['sentences'] = sentences[:12*max_batches*batch_size]

In [15]:
def get_new_candidates(classifier, positives, encoded_inputs, batch_size = 256, num_examples = 32000, max_positive=25):
    shuffled_index = np.random.permutation(encoded_inputs['input_ids'].size(0))
    for key, value in encoded_inputs.items():
        encoded_inputs[key] = value[shuffled_index] if isinstance(value, Tensor) else [value[i] for i in shuffled_index]
    
    corpus_dataset = TensorDataset(encoded_inputs['input_ids'], encoded_inputs['attention_mask'])
    corpus_dataloader = DataLoader(corpus_dataset, batch_size=batch_size, shuffle=False)
    scores, tokens = models.score_corpus(classifier, corpus_dataloader, max_positive=max_positive, max_batches=num_examples//batch_size, threshold=0.5)
    hits = list(zip(scores, tokens, encoded_inputs['sentences'][:len(scores)]))
    subset = [(score, token, sample) for score, token, sample in hits if
         score > 0.5 and not sample in positives]
    return [sample for _, _, sample in subset]

## Auto training flow

In [35]:
# sample a rule
rule = egp.sample(1).iloc[0]
print(f"""{rule['guideword']}: {rule['Can-do statement']}
{rule['Example']}""")
# ask GPT for search strings
words = get_words(rule)
print(words)
# ask GPT for detection questions
questions = get_questions(rule)
print(questions)
# do a sanity check of the questions with the given examples
positives, negatives = judge(rule['Example'].split("\n\n"), questions)
assert len(negatives)==0
# Find candidates in corpus
results, sorted_sentences = search_corpus(words, sents)
others = list(sents.difference(set([key for key, value in results.items() if value == 0])))
print(sorted_sentences[:10])
# judge candidates
positives = []
negatives = []

candidates = sorted_sentences[:50]
#random.shuffle(candidates)
candidates = list(set(candidates).difference(set(positives+negatives)))
new_positives, new_negatives = judge(candidates, questions)
assert len(new_positives) > 5
positives = positives + new_positives
negatives = negatives + new_negatives
 
# iteratively train classifier
metrics = {"precision": 0.}
iterations = 0
while metrics['precision'] < 0.8 and iterations < 5:
    iterations += 0.2
    classifier = models.RuleDetector(models.bert_encoder).to(models.device)
    classifier, metrics = get_trained_classifier(positives, negatives, others, classifier, ratio=3, verbose=True)
    print(metrics)
    if metrics['precision'] == 0.: continue
    # get new candidates
    new_candidates = get_new_candidates(classifier, positives, encoded_inputs)
    assert len(new_candidates) > 5
    iterations += 0.8
    # retrain on new candidates
    new_positives, new_negatives = judge(new_candidates, questions)
    precision = len(new_positives) / (len(new_positives)+len(new_negatives))
    print(precision)
    if precision > 0.75: break
    
    positives = positives + new_positives
    negatives = negatives + new_negatives

if metrics['f1']>0.75:
    models.save_classifier(classifier, rule['#'], "auto_training")

FORM: NEGATIVE: Can use the negative form with an increasing range of verbs.
I didn't realise I had lost the ring till I was at home. 

I didn't trust him. 

[talking about a spare television] ? we had one in the garage that we did not use.
["didn't", 'did not']
1. Does the sentence use the past simple tense?
2. Is the sentence in the negative form?
3. Does the sentence include a verb in its negative form?
4. Is the verb used in the sentence part of an increasing range of verbs (i.e., not limited to basic verbs like "be" or "have")?
5. Does the sentence make sense and convey a clear past action or state that did not happen?


  0%|          | 0/3 [00:00<?, ?it/s]

I didn't realise I had lost the ring till I was at home. 
1. Yes
2. Yes
3. Yes
4. Yes
5. Yes
I didn't trust him. 
1. Yes
2. Yes
3. Yes
4. Yes
5. Yes
[talking about a spare television] ? we had one in the garage that we did not use.
1. Yes
2. Yes
3. Yes
4. Yes
5. Yes


  0%|          | 0/451036 [00:00<?, ?it/s]

["I didn't dare look up at it as they advised because I did not want to hurt my eyes.", 'My father made varsity in high school football but did not persue it when he moved to Canada.', 'She did not have to think to hard for a name for that one, lol.', 'okay thank you i did not know that, however I hike in the desert how big are the bears there?', 'I did not do anything that sophisticated.', 'That is very interesting, I did not know that.', 'With MySpace, people who did not go out much could reach out to others from the comfort of their own homes.', 'closer to town it seemed crowded but once you go out in the country, it did not seem crowded', 'that i did not know', 'I have heard of the grand rapids, but I did not know they were in Michigan, is it free to go there?']


...

Training loss: 0.44165727496147156


Val loss: 0.4872833862900734
{'accuracy': 0.821, 'f1': 0.0, 'precision': 0.0}
78
0.1780821917808219


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.36it/s]
                                                                                                                                                                           

Training loss: 0.7644012570381165


KeyboardInterrupt: 

In [33]:
negatives

['When shall we have a coaching period?',
 'When shall we sign the contract, Mrs. Brown?',
 'Where shall we sit?',
 'What shall we do this weekend?',
 'When and where shall we meet?',
 'And by the way, what time shall we get to Nanchang?',
 'When shall we go?',
 'What shall we do when we find a shortage in the shipment?',
 'So, where shall we have our dinner?',
 'Where shall we park our car?',
 "I haven't been in his house for several weeks.When shall we start?",
 'Where shall we go?',
 "We're sleeping here tonight, Jason.",
 'A bunch of us are going there for dinner tomorrow night.How about coming with us?',
 'Jane, yesterday, I got a call from the local police station to pick up Biggie.',
 'I hope to go to one one day.',
 'And I hope I can enter your school one day.',
 "Mom, I don't want to bathe today.",
 "We'll have them worked out by this evening and let you have them tomorrow morning.",
 'It lasts 4 hours, from 10:00 to 14:00',
 'OK, I want 5 tickets for the 9 p.m. showing tonigh

In [26]:
# iteratively train classifier

iterations = 0
while metrics['f1'] < 0.75 and iterations < 5:
    iterations += 0.2
    classifier = models.RuleDetector(models.bert_encoder).to(models.device)
    classifier, metrics = get_trained_classifier(positives, negatives, others, classifier, ratio=3, verbose=False)
    print(metrics)
    if metrics['precision'] == 0.: continue
    # get new candidates
    new_candidates = get_new_candidates(classifier, positives, encoded_inputs)
    assert len(new_candidates) > 5
    iterations += 0.8
    # retrain on new candidates
    new_positives, new_negatives = judge(new_candidates, questions)
    if len(new_positives) / (len(new_positives)+len(new_negatives)) > 0.75: break
    
    positives = positives + new_positives
    negatives = negatives + new_negatives

if metrics['f1']>0.75:
    models.save_classifier(classifier, rule['#'], "auto_training")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.63it/s]


{'accuracy': 0.783, 'f1': 0.0, 'precision': 0.0}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.85it/s]


{'accuracy': 0.797, 'f1': 0.0, 'precision': 0.0}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.83it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.86it/s]


{'accuracy': 0.8, 'f1': 0.105, 'precision': 0.333}


  1%|█▍                                                                                                                                    | 2/192 [00:00<01:22,  2.30it/s]


  0%|          | 0/28 [00:00<?, ?it/s]

But I think you can fly on Northwest 212 to Tokyo and they have a connecting flight on Japan Airline 123 to Auckland, New Zealand's gateway city.
1. Yes
2. No
3. No
4. Yes
5. Yes
I think that a vegetarian diet is often thought to be lower in fat and cholesterol.
1. No
2. No
3. No
4. Yes
5. Yes
Yes there is skiing and you've also got the lake that forms half of Vermont's western border with New York.
1. No
2. No
3. No
4. Yes
5. Yes
Why not buy a guidebook to the city?
1. Yes
2. Yes
3. No
4. Yes
5. Yes
The book is really good.
1. No
2. No
3. No
4. Yes
5. Yes
If wealthy Americans talk about the unique challenges that come with their wealth, people often dismiss their experience.
1. Yes
2. No
3. No
4. Yes
5. Yes
taco predates the arrival of the Spanish in Mexico, there is anthropological evidence that the indigenous people living in the lake region of the Valley of Mexico traditionally ate tacos filled with small fish
1. No
2. No
3. No
4. No
5. Yes
Do you know much about bipolar disorder, 

In [27]:
len(new_positives) / (len(new_positives)+len(new_negatives))

0.07142857142857142

In [545]:
#words += [c[0] + " " + c[1] for c in combinations(words, 2) if c[0].count(' ')==0 and c[1].count(' ')==0] + [c[1] + " " + c[0] for c in combinations(words, 2) if c[0].count(' ')==0 and c[1].count(' ')==0]
words

["I've been",
 'I have been',
 "She's been",
 'She has been',
 "He's been",
 'He has been',
 "We've been",
 'We have been',
 "They've been",
 'They have been',
 "You've been",
 'You have been',
 'for',
 'since']

In [507]:
# output dataset

output_path = '../data/auto_annotated_corpus.json'
instances = pd.DataFrame(columns=['#', 'sentence', 'positive']) if not os.path.exists(output_path) else pd.read_json(output_path)
instances['positive'] = instances['positive'].astype(bool)

In [511]:
for sentence in positives:
    new_row = pd.DataFrame({'#': [rule['#']], 'sentence': [sentence], 'positive': [False]})
    instances = pd.concat([instances, new_row], ignore_index=True)
for sentence in negatives:
    new_row = pd.DataFrame({'#': [rule['#']], 'sentence': [sentence], 'positive': [False]})
    instances = pd.concat([instances, new_row], ignore_index=True)

In [514]:
instances.to_json(output_path)